In [ ]:
import pandas as pd
import json
import numpy as np
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import os
import plotly.io as pio
# The bin folder has the DLLs
os.environ['path'] += r';C:/Users/JKIM4/Downloads/vips-dev-w64-all-8.11.0/vips-dev-8.11/bin'
import pyvips
import requests

In [ ]:
# setting directories
dir_code = "../run"
dir_data = "../run/data"
dir_results = "../run/results"

# setting configs json file path
file_configs = dir_code + "/configs.json"


# reading configs json file
print("reading configuration json file from = {}".format(file_configs))
with open(file_configs, "r") as read_file:
    configs = json.load(read_file)
    
configs['dir_results'] = dir_results
configs['dir_data'] = dir_data
    
configs

In [ ]:
################################################################################################
#----------------------------------------------------------------------------------------------#
################################################################################################

def gas_rate(site_id, startyear, endyear):

    headers = {'Content-type': 'application/json'}
    data = json.dumps({"seriesid": [site_id],"startyear":startyear, "endyear":endyear})

    p = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    df_gas = pd.DataFrame()
    
    print(json_data)
    
    for series in json_data['Results']['series']:

        i = 0
        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','

            if 'M01' <= period <= 'M12':
                df_gas.at[i,'year'] = year
                df_gas.at[i,'period'] = period
                df_gas.at[i,'value'] = value
            i = i+1

    years = df_gas['year'].unique()          

    for year in years:
        df_gas_filtered = df_gas.loc[df_gas['year']==year]
        if len(df_gas_filtered) == 12:
            break
    return df_gas_filtered

################################################################################################
#----------------------------------------------------------------------------------------------#
################################################################################################

def utilRates(df,utility,sector,name):  # Electric

    df_filtered = df.loc[df['utility'] ==utility]
    df_filtered = df_filtered.loc[df_filtered['sector'] ==sector]
    df_filtered = df_filtered.loc[df_filtered['name'] == name]
    df_filtered['startdate'] = pd.to_datetime(df_filtered['startdate'], format='%m/%d/%Y %H:%M')
    df_filtered.sort_values(by='startdate')
    df_final = df_filtered.iloc[-1:].reset_index()
    cols = df_final.columns

    # filtering energy rates
    energyrates = [col for col in cols if 'energyratestructure' in col ]
    df_energyrates = df_final[energyrates]
    df_energyrates=df_energyrates.dropna(axis=1)

    # filtering demand rates
    demandrates = [col for col in cols if 'demandratestructure' in col ]
    df_demandrates = df_final[demandrates]
    df_demandrates=df_demandrates.dropna(axis=1)

    df_fixed_rate = df_final['fixedchargefirstmeter'] 

    schedules = ['demandweekdayschedule', 'demandweekendschedule', 'energyweekdayschedule', 'energyweekendschedule']
    for schedule in  schedules:  

        demandSchedule_updated= df_final[schedule].str.split(",")
        demandSchedule_updated_1 =[ str(row).replace("L","") for row in demandSchedule_updated.values]
        demandSchedule_updated_1 =[ str(row).replace("]","") for row in demandSchedule_updated_1]
        demandSchedule_updated_1 =[ str(row).replace("[","") for row in demandSchedule_updated_1]
        demandSchedule_updated_1 =[ str(row).replace("'","") for row in demandSchedule_updated_1]
        demandSchedule_updated_final= demandSchedule_updated_1[0].split(",")

        df_periods = pd.DataFrame()
        for i in range(0, len(demandSchedule_updated_final)):
            month = math.floor(i/24) + 1
            hrs = i - math.floor(i/24)*24
            df_periods.at[month,'hr'+str(hrs)] = int(demandSchedule_updated_final[i])
        df_periods.index.names = ['Months']
        if schedule == 'demandweekdayschedule':
            df_periods_demand_weekday = df_periods
        elif schedule == 'demandweekendschedule':
            df_periods_demand_weekend = df_periods
        elif schedule == 'energyweekdayschedule':
            df_periods_energy_weekday = df_periods
        elif schedule == 'energyweekendschedule':
            df_periods_energy_weekend = df_periods

    return [df_periods_demand_weekday, df_periods_demand_weekend, df_periods_energy_weekday, df_periods_energy_weekend,df_energyrates,df_demandrates,df_fixed_rate]  


In [ ]:
# # reading FDD results file
# df_result = pd.read_csv(configs["dir_results"] + "/{}_{}.csv".format( configs["weather"], configs["train_test_apply"] ))

# # creating empty dataframe with timestamp
# freq = str(configs['impact_est_timestep_min']) + 'min'
# df_combined = pd.DataFrame([])
# df_combined['reading_time'] = pd.date_range( configs['streamdata_date_start'], configs['streamdata_date_end'], freq=freq)
# df_combined = df_combined.set_index(['reading_time'])[:-1]
# timestamp_last = df_combined.index[-1]

# # expanding FDD results into the same user-specified timestep
# df_result_exp = np.repeat(list(df_result.values), configs['fdd_reporting_frequency_hrs']*int(60/configs['impact_est_timestep_min']))
# df_result_exp = pd.DataFrame(df_result_exp)
# df_result_exp.columns = ['FaultType']
# df_result_exp.index = df_combined.index

# # setting timestamp for raw simulation data
# freq_raw = str(configs['simulation_timestep_min']) + 'min'
# df_index = pd.DataFrame([])
# df_index['reading_time'] = pd.date_range( configs['simulation_date_start'], configs['simulation_date_end'], freq=freq_raw)
# df_index = df_index.set_index(['reading_time'])[:-1]

# # reading baseline simulation results
# print("[Estimating Fault Impact] reading baseline simulation results")
# df_baseline = pd.read_csv(configs['dir_data']+"/"+configs['weather']+"/baseline.csv", usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
# df_baseline.columns = [f"baseline_elec_{configs['sensor_unit_elec']}",f"baseline_ng_{configs['sensor_unit_ng']}"]
# df_baseline.index = df_index.index
# df_baseline = df_baseline.resample(str(configs['impact_est_timestep_min'])+"T").mean()

# # recreating FDD results with unique fault type (consecutive fault types are removed)
# df_unique = df_result_exp[(df_result_exp.ne(df_result_exp.shift())).any(axis=1)]
# df_unique = df_unique.reset_index()

# # reading individual fault simulation results (based on FDD results) and creating whole year combined results
# count = 1
# print("[Estimating Fault Impact] combining simulation results from the FDD results")
# df_combined_temp = pd.DataFrame()
# for index, row in df_unique.iterrows():

#     # specifying start and stop timestamp for each detected fault
#     rownum_current = df_unique.loc[df_unique.index==index,:].index[0]
#     timestamp_start = df_unique.iloc[rownum_current,:].reading_time
#     if rownum_current+1 < df_unique.shape[0]:
#         timestamp_end = df_unique.iloc[rownum_current+1,:].reading_time - pd.Timedelta(minutes=configs["simulation_timestep_min"])
#     else:
#         timestamp_end = timestamp_last

#     print(f"[Estimating Fault Impact] prossessing [{row['FaultType']} ({count}/{df_unique.shape[0]})] from the FDD results covering {timestamp_start} to {timestamp_end}")

#     count_file = 0
#     for file in glob.glob(configs['dir_data']+"/"+configs['weather']+f"/*{row['FaultType']}*"):
#         print(f"[Estimating Fault Impact] reading [{file}] file")
#         count_file += 1
#         if count_file == 1:
#             df_temp = pd.read_csv(file, usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
#             df_temp.index = df_index.index
#             df_temp = df_temp.resample(str(configs['impact_est_timestep_min'])+"T").mean()
#             df_temp = df_temp[timestamp_start:timestamp_end]
#             df_fault = df_temp.copy()
#         else:
#             df_temp = pd.read_csv(file, usecols=[configs['sensor_name_elec'], configs['sensor_name_ng']])
#             df_temp.index = df_index.index
#             df_temp = df_temp.resample(str(configs['impact_est_timestep_min'])+"T").mean()
#             df_temp = df_temp[timestamp_start:timestamp_end]
#             df_fault += df_temp

#     # averaging all fault intensity simulations for a single fault and merging into combined dataframe
#     print(f"[Estimating Fault Impact] averaging all fault intensity simulations for a single fault and merging into combined dataframe")
#     df_fault = df_fault/count_file
#     df_fault['fdd_result'] = row['FaultType']
#     df_combined_temp = pd.concat([df_combined_temp, df_fault])
#     count+=1

# # creating combined dataframe from baseline and faulted timeseries data
# df_combined_temp.columns = [f"faulted_elec_{configs['sensor_unit_elec']}",f"faulted_ng_{configs['sensor_unit_ng']}", "fdd_result"]
# df_combined_temp.index = pd.to_datetime(df_combined_temp.index)
# df_combined = pd.merge(df_combined, df_baseline, how='outer', left_index=True, right_index=True)
# df_combined = pd.merge(df_combined, df_combined_temp, how='outer', left_index=True, right_index=True)

# # creating columns of energy usage differences
# df_combined['diff_elec'] = df_combined["faulted_elec_{}".format(configs["sensor_unit_elec"])] - df_combined["baseline_elec_{}".format(configs["sensor_unit_elec"])]
# df_combined['diff_ng'] = df_combined["faulted_ng_{}".format(configs["sensor_unit_ng"])] - df_combined["baseline_ng_{}".format(configs["sensor_unit_ng"])]

# # creating columns for time, date, and month
# df_combined['Time'] = pd.to_datetime(df_combined.index).time
# df_combined['Time'] = df_combined.Time.astype(str).str.rsplit(":",1, expand=True).iloc[:,0]
# df_combined['Date'] = pd.to_datetime(df_combined.index).date
# df_combined['Month'] = pd.to_datetime(df_combined.index).month
# df_combined = df_combined.dropna()

# # calculate monthly and annual excess energy usages
# if (configs['sensor_unit_ng']=='W') & (configs['sensor_unit_elec']=='W'):
#     df_monthly = df_combined.groupby(['Month'])[["baseline_elec_{}".format(configs["sensor_unit_elec"]),"baseline_ng_{}".format(configs["sensor_unit_ng"]),'diff_elec','diff_ng']].sum()/1000/(60/configs['impact_est_timestep_min']) #convert W to kWh
#     base_annual_elec = round(df_monthly["baseline_elec_{}".format(configs["sensor_unit_elec"])].sum()) # in kWh
#     base_annual_ng = round(df_monthly["baseline_ng_{}".format(configs["sensor_unit_ng"])].sum()) # in kWh
#     diff_annual_elec = round(df_monthly.sum()['diff_elec']) # in kWh
#     diff_annual_ng = round(df_monthly.sum()['diff_ng']) # in kWh
#     perc_annual_elec = round(diff_annual_elec/base_annual_elec*100, 3) # in %
#     perc_annual_ng = round(diff_annual_ng/base_annual_ng*100, 3) # in %
# else:
#     # add other unit conversions
#     print("[Estimating Fault Impact] unit conversion from {} for electricity and {} for natural gas to kWh is not currently supported".format(configs['sensor_unit_elec'],configs['sensor_unit_ng']))



In [ ]:
# df_combined

In [ ]:
# df_combined.to_csv("./df_combined.csv")

In [ ]:
df_combined = pd.read_csv("./df_combined.csv", index_col=0)
df_combined

In [ ]:
df_combined.index = pd.to_datetime(df_combined.index,format='%Y-%m-%d %H:%M:%S')
df_combined.index

In [ ]:
# reading utility values from utility database
df_rates = pd.read_csv(configs['dir_data'] + '/cost_data/utility_data.csv')
[df_periods_demand_weekday, df_periods_demand_weekend, df_periods_energy_weekday, df_periods_energy_weekend,df_energyrates,df_demandrates,df_fixed_rate] = utilRates(df_rates, configs['rate_elec_utility'], configs['rate_elec_sector'], configs['rate_elec_name'])  
df_combined['baseline_ng_therms'] = df_combined['baseline_ng_W']* 0.034130/1000
df_combined['faulted_ng_therms'] = df_combined['faulted_ng_W']* 0.034130/1000   
df_combined = df_combined.reset_index()

# electricity cost estimation - energy 
for i in range(0,len(df_combined)):
    month = df_combined['reading_time'][i].month
    hr = df_combined['reading_time'][i].hour
    if df_combined['reading_time'][i].weekday() <= 4: #weekday
        columns = df_periods_energy_weekday.columns
        for col in columns:
            if str(hr) in col:
                col_needed = col
                break
        filtered_col_kwh = df_periods_energy_weekday[col_needed]
        rate_needed_kwh = filtered_col_kwh.loc[month]        
        df_combined.at[i,'rate_kwh'] = rate_needed_kwh
    else: # weekend
        columns = df_periods_energy_weekend.columns
        for col in columns:
            if str(hr) in col:
                col_needed = col
                break
        filtered_col_kwh = df_periods_energy_weekend[col_needed]
        rate_needed_kwh = filtered_col_kwh.loc[month]
        df_combined.at[i,'rate_kwh'] = rate_needed_kwh
    rate_cost = 0
    for col in df_energyrates.columns: 
        if 'period'+str(int(rate_needed_kwh)) in col:
            rate_cost = rate_cost + df_energyrates[col][0]
    df_combined.at[i,'baseline_elec_energy_cost_$'] = rate_cost * df_combined['baseline_elec_W'][i]/1000
    df_combined.at[i,'faulted_elec_energy_cost_$'] = rate_cost * df_combined['faulted_elec_W'][i]/1000    
    df_combined.at[i,'diff_elec_energy_cost_$'] = df_combined['faulted_elec_energy_cost_$'][i]-df_combined['baseline_elec_energy_cost_$'][i]

# gas cost estimation 
df_gas_rate = gas_rate(configs['rate_ng_siteid'], configs['rate_ng_year_start'], configs['rate_ng_year_end'])
df_gas_rate['Month'] = df_gas_rate.period.str.split("M", expand=True).iloc[:,1].astype(float)
df_combined = pd.merge(df_combined, df_gas_rate[['Month','value']], on='Month')
df_combined['value'] = df_combined['value'].astype(float)
df_combined = df_combined.rename(columns={'value':'rate_ng_$_per_therm'})
df_combined['baseline_ng_cost_$'] = df_combined.baseline_ng_therms * df_combined['rate_ng_$_per_therm']
df_combined['faulted_ng_cost_$'] = df_combined.faulted_ng_therms * df_combined['rate_ng_$_per_therm']
df_combined['diff_ng_cost_$'] = df_combined['faulted_ng_cost_$'] - df_combined['baseline_ng_cost_$']    

# electricity cost estimation - demand
df_combined_monthly = df_combined.groupby(by="Month").sum()
months = [1,2,3,4,5,6,7,8,9,10,11,12]
df_combined_monthly['rate_elec_subscription_cost_$'] = [df_fixed_rate[0]]*df_combined_monthly.shape[0]
cases = ['baseline_elec_W','faulted_elec_W']
for case in cases:
    title = case + '_demand_cost_$'
    for month in months:
        df_month = df_combined.loc[df_combined['Month'] == month]
        if df_month.shape[0] == 0:
            continue
        df_peak =df_month[df_month[case] == df_month[case].max()].reset_index()
        df_peak_hour = df_peak['reading_time'][0].hour
        if df_combined['reading_time'][i].weekday() <+ 4: #weekday
            columns = df_periods_demand_weekday.columns
            for col in columns:
                if str(df_peak_hour) in col:
                    col_needed = col
                    break
            filtered_col_demand = df_periods_demand_weekday[col_needed]     
            rate_needed_demand = filtered_col_demand.loc[month]
        else: # weekend
            columns = df_periods_demand_weekend.columns
            for col in columns:
                if str(df_peak_hour) in col:
                    col_needed = col
                    break
            filtered_col_demand = df_periods_demand_weekend[col_needed]     
            rate_needed_demand = filtered_col_demand.loc[month]   
        rate_cost = 0
        for col in df_demandrates.columns: 
            if 'period'+str(int(rate_needed_demand)) in col:
                rate_cost = rate_cost + df_demandrates[col][0]
        demand_cost = rate_cost * df_month[case].max()/1000
        df_combined_monthly.at[month,title] = demand_cost 

df_combined_monthly = df_combined_monthly.rename(columns={'baseline_elec_W_demand_cost_$':'baseline_elec_demand_cost_$', 'faulted_elec_W_demand_cost_$':'faulted_elec_demand_cost_$'})
df_combined_monthly['diff_elec_demand_cost_$'] = df_combined_monthly['faulted_elec_demand_cost_$'] - df_combined_monthly['baseline_elec_demand_cost_$']
df_combined_monthly['baseline_elec_cost_total_$'] = df_combined_monthly['baseline_elec_energy_cost_$'] + df_combined_monthly['baseline_elec_demand_cost_$'] + df_combined_monthly['rate_elec_subscription_cost_$']
df_combined_monthly['faulted_elec_cost_total_$'] = df_combined_monthly['faulted_elec_energy_cost_$'] + df_combined_monthly['faulted_elec_demand_cost_$'] + df_combined_monthly['rate_elec_subscription_cost_$'] 
df_combined_monthly['diff_elec_cost_$'] = df_combined_monthly['faulted_elec_cost_total_$'] - df_combined_monthly['baseline_elec_cost_total_$']

In [ ]:
df_combined_monthly

In [ ]:
df_impact = pd.DataFrame()
df_impact['impact_site_energy_elec_kWh'] = df_combined.groupby(['fdd_result']).diff_elec.sum()/1000 # in kWh
df_impact['impact_site_energy_ng_kWh'] = df_combined.groupby(['fdd_result']).diff_ng.sum()/1000 # in kWh
df_impact['fault_duration_ratio'] = df_combined.groupby(['fdd_result']).Date.count() / df_combined.shape[0]
df_impact = df_impact.reset_index()

In [ ]:
df_combined